In [72]:
import sys, os
import numpy as np
import matplotlib.pylab as plt
import glob
from Parameters import *

In [73]:
numMeas = 20
dimention = 11
nBin = 256
anglePerBin = 2*np.pi/nBin
AOAThreshold = anglePerBin

In [74]:
pythonFilePath = os.getcwd()
path = glob.glob(os.path.join(pythonFilePath, 'Datasets', '*'))

data = np.empty((0,dimention))
GT = []

In [75]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [76]:
model = tf.keras.models.load_model('256bin.h5')

In [77]:
X_test = np.empty((0,dimention))
y_test = []

npzfile = np.load(os.path.join(path[9], "processedData.npz"))

# extract CIR
realCIR = npzfile['realCIR'].astype(dtype=np.float32)
imagCIR = npzfile['imagCIR'].astype(dtype=np.float32)
offsetCIR = npzfile['offsetCIR'].astype(dtype=np.float32)
# extract AOA
AOA = npzfile['yaw_m_mc'].astype(dtype=np.float32)
# calculate magnitude
magCIR = np.sqrt(realCIR ** 2 + imagCIR** 2)
AOA = AOA + np.pi

In [78]:
for j in range(nBin):
    std = j*anglePerBin
    angleDiff = AOA-std
    angleDiff = np.where(angleDiff < 0, 2*np.pi, angleDiff)
    AOAThresholdMask = angleDiff<AOAThreshold
    selectionIndices = np.nonzero(AOAThresholdMask)[0][:]
        
    value = magCIR[selectionIndices,4:4+dimention]
    idx = np.ones(value.shape[0])*j
        
    X_test = np.append(X_test, value, axis = 0)
    y_test = np.append(y_test, idx)
    
y_test = y_test.reshape(-1, 1)

In [79]:
results = model.evaluate(X_test, y_test, batch_size = 2000)

106/106 [==============================] - 3s 24ms/step - loss: 5.1316 - accuracy: 0.0269


In [80]:
y_test.shape

(211923, 1)

In [81]:
test_data = {}
for i in range(nBin):
    test_data[i] = np.empty((0,dimention))

for k in range(y_test.shape[0]):
    temp = X_test[k,:].reshape(1, 11)
    test_data[int(y_test[k])] = np.vstack((temp, test_data[int(y_test[k])]))

In [82]:
numcase = 10
all_accuracy = 0

for i in range(nBin):
    pred_sample = model.predict(test_data[i])
    cnt = 0
    downswitch = 0
    upswitch = 0
    down_boundary = i - 10
    up_boundary = i + 10
    if down_boundary < 0:
        down_boundary = down_boundary + nBin
        downswitch = 1
    if up_boundary >= nBin:
        up_boundary = up_boundary - nBin
        upswitch = 1
    for j in range(int(test_data[0].shape[0]/numcase)):
        sum_data = np.zeros(nBin)
        sum_data = np.sum(pred_sample[j*numcase:(j+1)*numcase, :], axis = 0)
        probabilty = np.zeros(256)
        for k in range(256):
            if k-10 < 0:
                probabilty[k] = np.sum(sum_data[k-10+255:]) + np.sum(sum_data[:k+10])
            elif k+10 > 255:
                probabilty[k] = np.sum(sum_data[k-10:]) + np.sum(sum_data[:k+10-255])
            else:
                probabilty[k] = np.sum(sum_data[k-10:k+10])
        if (downswitch + upswitch) == 0:
            if down_boundary < np.argmax(probabilty) and np.argmax(probabilty) < up_boundary:
                cnt = cnt + 1
        elif downswitch == 1:
            if down_boundary < np.argmax(probabilty) or np.argmax(probabilty) < up_boundary:
                cnt = cnt + 1
        elif upswitch == 1:
            if down_boundary < np.argmax(probabilty) or np.argmax(probabilty) < up_boundary:
                cnt = cnt + 1
#         plt.bar(x, sum_data)
#         plt.xticks(x, index)
#         plt.title(i)
#         plt.show()
    print(i, 'accuracy : ', cnt/int(test_data[0].shape[0]/numcase)*100)
    all_accuracy = all_accuracy + cnt/int(test_data[0].shape[0]/numcase)*100
print('all accuracy', all_accuracy/nBin)

0 accuracy :  64.28571428571429
1 accuracy :  37.5
2 accuracy :  75.0
3 accuracy :  44.642857142857146
4 accuracy :  57.14285714285714
5 accuracy :  48.214285714285715
6 accuracy :  57.14285714285714
7 accuracy :  44.642857142857146
8 accuracy :  66.07142857142857
9 accuracy :  41.07142857142857
10 accuracy :  33.92857142857143
11 accuracy :  44.642857142857146
12 accuracy :  37.5
13 accuracy :  23.214285714285715
14 accuracy :  41.07142857142857
15 accuracy :  37.5
16 accuracy :  41.07142857142857
17 accuracy :  26.785714285714285
18 accuracy :  37.5
19 accuracy :  26.785714285714285
20 accuracy :  21.428571428571427
21 accuracy :  41.07142857142857
22 accuracy :  25.0
23 accuracy :  37.5
24 accuracy :  19.642857142857142
25 accuracy :  25.0
26 accuracy :  30.357142857142854
27 accuracy :  33.92857142857143
28 accuracy :  32.142857142857146
29 accuracy :  28.57142857142857
30 accuracy :  33.92857142857143
31 accuracy :  32.142857142857146
32 accuracy :  32.142857142857146
33 accuracy 

In [59]:
j = 10

In [60]:
sum_data = np.sum(pred_sample[j*numcase:(j+1)*numcase, :], axis = 0)

In [61]:
sum_data.shape

(256,)

In [66]:
sum_data

array([0.25043806, 0.18875384, 0.06509464, 0.05116528, 0.04889578,
       0.05382594, 0.04478561, 0.05875426, 0.05455231, 0.05827201,
       0.04820729, 0.03271614, 0.04518821, 0.06063749, 0.04834709,
       0.02274785, 0.02183405, 0.02098758, 0.01868018, 0.03369132,
       0.05925561, 0.03849781, 0.04936718, 0.03731075, 0.02548165,
       0.03308592, 0.03179514, 0.02910596, 0.02614862, 0.02121169,
       0.01467369, 0.02062478, 0.02197994, 0.01425797, 0.00784333,
       0.00842035, 0.01290274, 0.00983137, 0.00662988, 0.00909905,
       0.01631164, 0.01552842, 0.01069162, 0.01260841, 0.02731336,
       0.01022103, 0.00933717, 0.0185696 , 0.03188211, 0.01097291,
       0.01866682, 0.01147001, 0.01315148, 0.01130664, 0.01708077,
       0.01550854, 0.01285062, 0.01164096, 0.01517839, 0.01251825,
       0.01873872, 0.01955542, 0.03293841, 0.10519922, 0.0094372 ,
       0.01160494, 0.00900941, 0.00780919, 0.00593449, 0.00643541,
       0.00519298, 0.00775502, 0.00594528, 0.00647638, 0.00548

In [69]:
probabilty = np.zeros(256)
for k in range(256):
    if k-10 < 0:
        probabilty[k] = np.sum(sum_data[k-10+255:]) + np.sum(sum_data[:k+10])
    elif k+10 > 255:
        probabilty[k] = np.sum(sum_data[k-10:]) + np.sum(sum_data[:k+10-255])
    else:
        probabilty[k] = np.sum(sum_data[k-10:k+10])

In [70]:
probabilty

array([2.95158172, 2.98368835, 2.9884901 , 2.99369526, 2.93440962,
       2.79754233, 2.53898692, 2.28669548, 1.93025947, 1.60895121,
       1.22757506, 1.03639245, 0.88613641, 0.87040901, 0.85655439,
       0.83314031, 0.81240028, 0.79940987, 0.76976156, 0.74135792,
       0.70429754, 0.67076397, 0.65867257, 0.63546431, 0.5890848 ,
       0.548581  , 0.53425354, 0.5253222 , 0.51416606, 0.50211573,
       0.47752348, 0.43457949, 0.4116101 , 0.37293455, 0.34823221,
       0.35006386, 0.32719898, 0.30474102, 0.29420465, 0.29993817,
       0.28969941, 0.29369253, 0.28453773, 0.27570927, 0.27275798,
       0.28199539, 0.28908357, 0.28903145, 0.29084104, 0.29938954,
       0.30280876, 0.30523583, 0.30926284, 0.33150959, 0.4241004 ,
       0.40622425, 0.40760818, 0.40728042, 0.39651999, 0.37057236,
       0.36603484, 0.35256103, 0.34884599, 0.34163985, 0.33680958,
       0.32521862, 0.31551591, 0.30663607, 0.29936442, 0.28812668,
       0.27925488, 0.26445544, 0.25262898, 0.22510242, 0.14269

In [51]:
np.argmax(probabilty)

238